In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns




try:
    df = pd.read_csv("/kaggle/input/used-cars1/used_cars.csv")
except FileNotFoundError:
    print("Error: 'used_cars.csv' not found. Please check the file path.")
    exit()



if 'milage' in df.columns:
    df.rename(columns={'milage': 'mileage'}, inplace=True)


df['mileage'] = df['mileage'].astype(str).str.replace('mi.', '', regex=False)
df['mileage'] = df['mileage'].str.replace(',', '', regex=False)
df['mileage'] = pd.to_numeric(df['mileage'], errors='coerce')


df['price'] = df['price'].astype(str).str.replace('$', '', regex=False)
df['price'] = df['price'].str.replace(',', '', regex=False)
df['price'] = pd.to_numeric(df['price'], errors='coerce')


df = df.dropna(subset=['mileage', 'price'])


Q1_price = df['price'].quantile(0.25)
Q3_price = df['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
price_lower = Q1_price - 1.5 * IQR_price
price_upper = Q3_price + 1.5 * IQR_price

Q1_mileage = df['mileage'].quantile(0.25)
Q3_mileage = df['mileage'].quantile(0.75)
IQR_mileage = Q3_mileage - Q1_mileage
mileage_lower = Q1_mileage - 1.5 * IQR_mileage
mileage_upper = Q3_mileage + 1.5 * IQR_mileage

df = df[
    (df['price'] >= price_lower) & 
    (df['price'] <= price_upper) &
    (df['mileage'] >= mileage_lower) &
    (df['mileage'] <= mileage_upper)
]



X = df[['mileage']]
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


lr_predictions = lr_model.predict(X_test)


lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))
print(f"Linear Regression RMSE: ${lr_rmse:,.2f}")


plt.figure(figsize=(12, 8))
plt.scatter(X_test, y_test, alpha=0.5, color='blue', label='Actual Prices')
plt.scatter(X_test, lr_predictions, alpha=0.5, color='red', label='Linear Regression Predictions')
plt.xlabel('Mileage')
plt.ylabel('Price ($)')
plt.title('Actual vs. Predicted Prices (Linear Regression)')
plt.legend()
plt.grid(True)
plt.show()

k = 5  
knn_model = KNeighborsRegressor(n_neighbors=k)
knn_model.fit(X_train, y_train)


knn_predictions = knn_model.predict(X_test)


knn_rmse = np.sqrt(mean_squared_error(y_test, knn_predictions))
print(f"KNN Regression RMSE: ${knn_rmse:,.2f}")


plt.figure(figsize=(12, 8))
plt.scatter(X_test, y_test, alpha=0.5, color='blue', label='Actual Prices')
plt.scatter(X_test, knn_predictions, alpha=0.5, color='green', label=f'KNN Predictions (k={k})')
plt.xlabel('Mileage')
plt.ylabel('Price ($)')
plt.title('Actual vs. Predicted Prices (KNN Regression)')
plt.legend()
plt.grid(True)
plt.show()


print("\nModel Comparison:")
print("-" * 50)
print(f"Linear Regression RMSE: ${lr_rmse:,.2f}")
print(f"KNN Regression RMSE: ${knn_rmse:,.2f}")